In [4]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


In [5]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction

algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = "fFG6R4JTFq9Hh8RjZy130aMRdUhmOYp68hi9nm4L"
headers = {
   "X-API-Key": algod_token,
}

algod_client = algod.AlgodClient(algod_token, algod_address, headers);


In [6]:
funded_mnemonics= ["rookie similar wire owner wash once fish mosquito glad coffee family venture adult funny melt gas inspire tuna buzz sell dignity pottery gold able bracket",
                  "narrow mimic suffer top suspect follow menu broccoli try snake meat erase napkin lucky client forget sense bread glad eight uniform bacon code about crack", 
                  "city truth device clog grocery sea safe slide glove borrow swap capable trash shaft vast start space distance calm wire hub crush dose ability army", 
                  "unknown arctic antenna country credit december ill practice lawsuit today athlete rescue exit swarm fitness strong minimum soldier wide coffee vacuum piece coil absorb unable", 
                  "gravity adult destroy demand margin coast culture base adjust east banana certain happy daughter bless fiscal fiscal eye forget inspire brain banner evil able bonus"]

funded_accounts = [{'sk': mnemonic.to_private_key(memo), 'pk': mnemonic.to_public_key(memo)} for memo in funded_mnemonics]

In [7]:
def get_asset_info(account): 
    return algod_client.account_info(account['pk'])['assets']

In [8]:
get_asset_info(funded_accounts[1])

[{'amount': 42,
  'asset-id': 13164495,
  'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
  'is-frozen': False}]

In [9]:
creator_passphrase = "angry live wrist arrest nerve stadium round candy scrap area road defy grow panic turkey attack moon shadow they worth shoe forum rural ability average"
creator_account = {'sk': mnemonic.to_private_key(creator_passphrase), 'pk': mnemonic.to_public_key(creator_passphrase)}

## Smart Contract 

In [7]:
from pyteal import *    
    
def approval_program():     
    on_creation = Seq([    
        App.globalPut(Bytes("Creator"), Txn.sender()),    
        App.globalPut(Bytes("Name"), Txn.application_args[0]),
        App.globalPut(Bytes("asset_id"), Btoi(Txn.application_args[1])),  # should be Int 
        Return(Int(1))
        # TODO save voting end time 
    ])    
    
    on_closeout = Return(Int(1))
    
    asset_id = App.globalGet(Bytes("asset_id"))
    # parameter indicates index in accounts array (0 is sender)     
    asset_balance = AssetHolding.balance(Int(0), asset_id)    
        
    on_optin = Seq([     # TODO if not before voting start time 
        asset_balance, 
        If(asset_balance.hasValue(),  # is sender has balance, store it, otherwise return 0     
           Seq([App.localPut(Int(0), Bytes("QVoteDecisionCredits"), asset_balance.value()), Return(Int(1))]),      
           Return(Int(0))
        ) 
    ])  
    
    # TODO there must be a better way of storing choices. Right now each voter can simply add a new choice by voting on a new name. 
    # choices should be fixed in the smart contract at creation. 
    choice = Txn.application_args[1]
    choice_tally = App.globalGet(choice)
    votes = Btoi(Txn.application_args[2])
    credit_balance = App.localGet(Int(0), Bytes("QVoteDecisionCredits"))
    credits_left = credit_balance - (votes*votes)
    on_vote = Seq([   # TODO check if voting has started 
        If(credits_left, 
            Seq([
                App.localPut(Int(0), Bytes("QVoteDecisionCredits"), credits_left), 
                App.globalPut(choice, choice_tally+votes),
                Return(Int(1))
            ]),
           Return(Int(0))
        )
    ])
        
    # IF VOTING     
    # check balance of sender,     
    # add votes     
    # remove square of votes from asset balance for that address     
        
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("vote"), on_vote]
    )    
    return program    
    
    
# TODO add timeframes for opt in and voting     
    
def clear_state_program():     
    program = Return(Int(1))
    return program    
    


In [62]:
# TODO 
# ADD VOTE START TIME 
# TEST MULTIPLE VOTES ON MULTIPLE CANDIDATES 
# FIND UPPER BOUNDS AND PARAMETERS FOR MULTIPLE CANDIDATES 

## Compile and deploy 

In [8]:
approval_filename = 'approval.teal'
clear_state_filename = 'clear_state.teal'

# pyteal to teal 
with open(approval_filename, 'w') as f:
    compiled = compileTeal(approval_program(), Mode.Application)
    f.write(compiled)

with open(clear_state_filename, 'w') as f:
    compiled = compileTeal(clear_state_program(), Mode.Application)
    f.write(compiled)

# teal to bytecode 
stdout, stderr = execute(["goal", "clerk", "compile", "-o", approval_filename+'.tok', approval_filename])
if stderr != "":
    print(stderr)
    raise
elif len(stdout) < 59:
    print("error in compile teal")
    raise
    
stdout, stderr = execute(["goal", "clerk", "compile", "-o", clear_state_filename+'.tok', clear_state_filename])
if stderr != "":
    print(stderr)
    raise
elif len(stdout) < 59:
    print("error in compile teal")
    raise
    
with open(approval_filename+'.tok', 'rb') as f:
    approval_bytes = f.read()
    
with open(clear_state_filename+'.tok', 'rb') as f: 
    clear_state_bytes = f.read() 

In [10]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema, OnComplete

In [10]:
asset_id = 13164495

params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

decision_name = 'to_be_or_not_to_be'

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=3, num_byte_slices=3)

on_complete = OnComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    funded_accounts[2]['pk'], 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
    app_args = [decision_name.encode('utf-8'), asset_id.to_bytes(3, 'big')]
)

app_create_txn_signed = app_create_txn.sign(funded_accounts[2]['sk'])
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'6VTR5HQCC3UAH7OV3WQTJBMWRPEKZO3Q4VYT3LW7WBKZ2IDDBBAA'

In [13]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
app_id

14201605

In [12]:
app_id = 14201605

## Testing vote

### Opt in 

In [13]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

# funded_accounts[1] has some tokens 
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[1]['pk'], params, app_id)
optin_txid = algod_client.send_transaction(optin_tx.sign(funded_accounts[1]['sk']))

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction 327JFIBQIL2FRXEXPOXYYW534W3ABGVMZHBW6CRVWLFVB4346Z7Q: account WSBV354QUX5TBQ7KNBRVCRMAFXJOJLKFIX46DIO2CP5BB7VQPHTCOKNGHQ has already opted in to app 14201605"}


In [14]:
optin_tx_response = algod_client.pending_transaction_info(optin_txid)
optin_tx_response

NameError: name 'optin_txid' is not defined

In [15]:
# local storage contains the election credits 
local_storage = algod_client.account_info(funded_accounts[1]['pk'])['apps-local-state']
local_storage

[{'id': 13199520,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 7}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14201605,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 42}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 13166035,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 42}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

### Decoding 

In [16]:
v = local_storage[-1]['key-value'][0]['key']

In [20]:
import base64
print(base64.b32encode(bytearray("abc", 'ascii')).decode('utf-8'))

MFRGG===


In [21]:
import base64
base64.b64encode("QVoteDecisionCredits".encode('utf-8')).decode('utf-8')

'UVZvdGVEZWNpc2lvbkNyZWRpdHM='

In [22]:
base64.b64decode(v).decode('utf-8')

'QVoteDecisionCredits'

## Valid votes 

In [31]:
# call application with arguments
votes = 3
app_args = ["vote".encode("utf-8"), "Zaphod".encode("utf-8"), votes.to_bytes(2, "big")]

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(funded_accounts[1]['pk'], params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[1]['sk']))

In [32]:
call_tx_response = algod_client.pending_transaction_info(call_txid)
call_tx_response 

{'confirmed-round': 10665220,
 'global-state-delta': [{'key': 'WmFwaG9k',
   'value': {'action': 2, 'uint': 3}}],
 'local-state-delta': [{'address': 'WSBV354QUX5TBQ7KNBRVCRMAFXJOJLKFIX46DIO2CP5BB7VQPHTCOKNGHQ',
   'delta': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
     'value': {'action': 2, 'uint': 33}}]}],
 'pool-error': '',
 'txn': {'sig': 'gTD8+stTqu/OMO4M4Trq+4fkPCZGmK7vXLC7w7jr7l0XHVWLIKxvDcp9KFytbY5Q2eeyTjT1pb1o2Hm28bgSDg==',
  'txn': {'apaa': ['dm90ZQ==', 'WmFwaG9k', 'AAM='],
   'apid': 13199520,
   'fee': 235000,
   'fv': 10664964,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 10665964,
   'snd': 'WSBV354QUX5TBQ7KNBRVCRMAFXJOJLKFIX46DIO2CP5BB7VQPHTCOKNGHQ',
   'type': 'appl'}}}

### Global storage - has the vote been counted 

In [14]:
# read app global state
def read_global_state(client, addr, app_id):   
    results = client.account_info(addr)
    apps_created = results['created-apps']
    global_states = []
    for app in apps_created :
        if app['id'] == app_id :
            global_states.append(app['params']['global-state'])
            # print(f"global_state for app_id {app_id}: ", app['params']['global-state'])
    return global_states 

In [15]:
## check global state for updates in votes 
read_global_state(algod_client, funded_accounts[2]['pk'], app_id)

[[{'key': 'YXNzZXRfaWQ=', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}},
  {'key': 'Q3JlYXRvcg==',
   'value': {'bytes': '/YwCdgGpi2dAH3N8r0McGQR3y/ICpC8JgRo0IzwoxF8=',
    'type': 1,
    'uint': 0}},
  {'key': 'TmFtZQ==',
   'value': {'bytes': 'dG9fYmVfb3Jfbm90X3RvX2Jl', 'type': 1, 'uint': 0}}]]

In [16]:
"hello".encode('utf-8')

b'hello'

### Local storage  - the user has spent credits 

In [39]:
# did anything actually change in the local storage? 
local_storage = algod_client.account_info(funded_accounts[1]['pk'])['apps-local-state']
local_storage

[{'id': 13199520,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 33}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 13166035,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 42}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

### More voting 

In [54]:
# voting again 
# call application with arguments
votes = 5
app_args = ["vote".encode("utf-8"), "Zaphod".encode("utf-8"), votes.to_bytes(2, "big")]   # same candidate 

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(funded_accounts[1]['pk'], params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[1]['sk']))

In [55]:
## check global state for updates in votes 
read_global_state(algod_client, funded_accounts[2]['pk'], app_id)

[[{'key': 'YXNzZXRfaWQ=', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}},
  {'key': 'Q3JlYXRvcg==',
   'value': {'bytes': '/YwCdgGpi2dAH3N8r0McGQR3y/ICpC8JgRo0IzwoxF8=',
    'type': 1,
    'uint': 0}},
  {'key': 'WmFwaG9k', 'value': {'bytes': '', 'type': 2, 'uint': 8}},
  {'key': 'TmFtZQ==',
   'value': {'bytes': 'dG9fYmVfb3Jfbm90X3RvX2Jl', 'type': 1, 'uint': 0}}]]

In [56]:
# did anything actually change in the local storage? 
local_storage = algod_client.account_info(funded_accounts[1]['pk'])['apps-local-state']
local_storage

[{'id': 13199520,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 8}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 13166035,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 42}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

### Invalid votes - more votes than allowed 

In [60]:
# voting again 
# call application with arguments
votes = 15
app_args = ["vote".encode("utf-8"), "Zaphod".encode("utf-8"), votes.to_bytes(2, "big")]   # same candidate 

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(funded_accounts[1]['pk'], params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[1]['sk']))

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction TOMLWPZVZFPWAY4EG2RQ7D2QHXEEVGSX6T2HEHPHZPHH42T2JY3A: - would result negative"}


In [61]:
# did anything actually change in the local storage?   
local_storage = algod_client.account_info(funded_accounts[1]['pk'])['apps-local-state']
local_storage

[{'id': 13199520,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 7}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 13166035,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 42}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]